<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/NOVA_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read intake data (including Descriptionen and FoodGroupen)
import pandas as pd
import re

file_path = "/content/drive/MyDrive/UPF-HFI/Bradford_original data/1. Dietmasterfile_foodlevel_clean.xls"
intake_df = pd.read_excel(file_path)

# Define text cleaning function
def clean_text(col):
    return col.astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# Apply to key fields
intake_df["Foodgroupen_clean"] = clean_text(intake_df["Foodgroupen"])
intake_df["Descriptionen_clean"] = clean_text(intake_df["Descriptionen"])

# Add a cleaning column for Subgroupcode
intake_df["Subgroupcode_clean"] = clean_text(intake_df["Subgroupcode"])


# Match foodgroups that already have NOVA1-6

In [ ]:
# att3
att3_path = "/content/drive/MyDrive/UPF-HFI/nova matching files/att3-excel.xlsx"

# read
att3 = pd.read_excel(att3_path)

# Cleaning group code
att3['code_clean'] = att3['Subsidiary food group code'].astype(str).str.upper().str.strip()

# Create a dictionary (including *)
group_to_nova = dict(zip(att3['code_clean'], att3['NOVA food group']))

# Clean the Subgroupcode column of the intake data
intake_df['Subgroupcode_clean'] = intake_df['Subgroupcode'].astype(str).str.upper().str.strip()


In [ ]:
# Map group code to NOVA_step1
intake_df['NOVA_step1'] = intake_df['Subgroupcode_clean'].map(group_to_nova)

# Add matching description column
intake_df['match_reason_step1'] = intake_df['NOVA_step1'].apply(
    lambda x: 'group code match' if str(x).isdigit() else None
)

# Matching Preview
intake_df['NOVA_step1'].value_counts(dropna=False)


,count
NOVA_step1,
4,8626
*,7643
1,4360
3,483
**,414
NaN,356
2,335


In [ ]:
# Keep key columns to export check files
cols_to_export = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]

check_df = intake_df[cols_to_export].copy()
check_df.to_excel("/content/drive/MyDrive/UPF-HFI/NOVA classification/outcome/step1.xlsx", index=False)


Step 2: Manual Processing
Processing Logic: Manual processing according to Dicken rule documentation - Lisa verification - Approval

In [ ]:
# Read the table from step 2
step2_path = "/content/drive/MyDrive/UPF-HFI/NOVA classification/outcome/step2.xlsx"
step2_df = pd.read_excel(step2_path)


In [ ]:
# Create a dictionary to extract the Descriptionen -> NOVA_step2 mapping from step2_df
desc_to_nova2 = step2_df.set_index("Descriptionen")["NOVA_step2"].to_dict()
desc_to_reason2 = step2_df.set_index("Descriptionen")["match_reason_step2"].to_dict()

# Map directly into intake_df
intake_df["NOVA_step2"] = intake_df["Descriptionen"].map(desc_to_nova2)
intake_df["match_reason_step2"] = intake_df["Descriptionen"].map(desc_to_reason2)


In [ ]:
# Make an intake24 dataset including NOVA
intake_df['NOVA'] = intake_df['NOVA_step2'].combine_first(intake_df['NOVA_step1'])


# Check

In [ ]:
intake_df['NOVA'].value_counts(dropna=False)


,count
NOVA,
4,10660
1,9186
3,997
2,960
NC,414


In [ ]:
# Export the entire intake table
intake_df.to_excel(
    "/content/drive/MyDrive/UPF-HFI/NOVA classification/outcome/intake24_with_nova.xlsx",
    index=False
)
